# CS 4501 Algorithmic Economics - Project 1

**Note:** For each of the question, please add some print or graph drawing commands to show your results in a clear way and also necessary analyses and demonstrations to help people who are not in your group understand your logics and results.


## Part 1
### Question 1
Using a Jupyter notebook import the csv file as pandas dataframe.

In [300]:
import pandas as pd
import numpy as np
import time as time
from tqdm import tqdm
from sklearn import linear_model as lm
df = pd.read_csv('./scanner_data.csv', index_col=0) # Tell pandas to ignore index column

# Note: data format is DD/MM/YYYY and starts at 2/1/2016
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

print(  "Number of Customer_ID",    len(pd.unique(df['Customer_ID']))  )
print(  "Number of SKU  ",    len(pd.unique(df['SKU']))  )
print(max(df['Date']))

Number of Customer_ID 22625
Number of SKU   5242
2016-12-31 00:00:00


### Question 2
The fact that consumer does not purchase anything can be 
interpreted as that she chose an outside option. Given that the fact that she chose an outside option is not recorded in this dataset, argue how you would construct a proxi variable for the choice of an outside option. Add such a proxi variable to your dataframe. 

**Hint:** you can use information that some consumers do not appear in the data every week. 

**Please input your answer in this cell:**




Add such a proxi variable to your dataframe. 


In [429]:
#Filter the data
df = df.sort_values(by=['SKU'])
SKUIndices = {}
SKU = pd.unique(df['SKU'])
for i in tqdm(range(len(SKU))):
    SKUIndices[SKU[i]] = i;
    
customers = pd.unique(df['Customer_ID'])
numCust = len(customers)

numProd = len(SKU)
utilities = np.zeros((numCust, numProd))

for i in tqdm(range(numCust)):
    custArr = df[df['Customer_ID'] == customers[i]]
    utilities[i,:] = 0
    prodIDs = custArr['SKU'].unique()
    for j in range(len(prodIDs)):
        utilities[i,SKUIndices[prodIDs[j]]] = utilities[i,SKUIndices[prodIDs[j]]] + len(custArr[custArr['SKU'] == SKU[SKUIndices[prodIDs[j]]]].Date.dt.isocalendar().week.unique())
    


100%|████████████████████████████████████| 22625/22625 [02:43<00:00, 138.69it/s]


In [297]:
print(len(custArr[custArr['SKU'] == SKU[SKUIndices[prodIDs[j]]]].Date.dt.isocalendar().week.unique()))

1


### Question 3
Given that we do not have **explicit** consumer feature vectors $\mathbf{x}^i = (x^i_1, \cdots, x_k^i)$  in the data, discuss how you would construct such feature vectors for each consumer $i$ from the given data. Add your constructed characteristics to your dataframe. 

**Hint:** you can use transaction history and argue that past shopping patterns may give a good characterization for a given consumer.

**Please input your answer in this cell:**





Add your constructed characteristics to your dataframe. 

In [444]:

featureCount = 7
custFeatureArr = np.zeros((numCust,featureCount))

for i in tqdm(range(numCust)):
    # First Organize Data & Calculate Features
    transactions = df[df.Customer_ID == customers[i]]
    custFeatureArr[i,0] = transactions.Quantity.sum()
    custFeatureArr[i,1] = transactions.Sales_Amount.sum()
    custFeatureArr[i,2] = custFeatureArr[i,1]/len(transactions.Date.unique())
    custFeatureArr[i,3] = len(transactions.Date.unique())
    custFeatureArr[i,4] = len(transactions.SKU.unique())
    custFeatureArr[i,5] = len(transactions.SKU_Category.unique())
    custFeatureArr[i,6] = len(transactions.Date.dt.isocalendar().week.unique())

prices = np.zeros((numProd,1))
for i in tqdm(range(numProd)):
    prices[i,0] = (df[df['SKU'] == SKU[i]].Sales_Amount / df[df['SKU'] == SKU[i]].Quantity).iloc[0]
    

  7%|██▊                                     | 374/5242 [00:12<02:39, 30.58it/s]


KeyboardInterrupt: 

[2 0 0 0 0]
[0 1 2]
[[9.95844568e-01 2.34071061e-03 5.00258389e-04 8.28739306e-04
  4.85723293e-04]
 [9.99710977e-01 2.66411860e-04 6.98571792e-06 1.01299132e-05
  5.49583269e-06]
 [9.99999994e-01 6.33021641e-09 9.50965232e-13 1.79266724e-12
  1.36410840e-12]
 [9.98071947e-01 1.20102044e-03 2.23058072e-04 2.70983020e-04
  2.32991394e-04]
 [9.99674869e-01 2.69863145e-04 1.68743572e-05 2.15153005e-05
  1.68783379e-05]]
6
0


### Question 4
Produce the utility parameters $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$ for every product $j$  by estimating a multinomial 
logit model from your constructed dataset.

In [445]:
#Hint: you can use sklearn.linear_model.LogisticRegression() to achieve an estimation
shuffledArr = np.concatenate((custFeatureArr, utilities),axis=1)
np.random.shuffle(shuffledArr)

numRuns = 2

scores = np.zeros((numRuns,1))
X = shuffledArr[:,0:featureCount]
utilities = shuffledArr[:,featureCount:-1]
X = np.append(X,np.zeros((numCust,1)),axis=1)
X = np.append(X,np.ones((numCust,1)),axis=1)

training = int(len(X))

for i in tqdm(range(numRuns)):
    X[:,featureCount] = -1 * prices[i]
    y = utilities[:,i].astype(int)
    y_train = y[0:training]
    x_train = X[0:training,:]
    y_test = y[training:-1]
    x_test = X[training:-1,:]
    lm.LogisticRegression(multi_class='multinomial')
    model = lm.LogisticRegression().fit(x_train, y_train)
    scores[i,0] = model.score(x_train,y_train)
    print(model.predict(x_train).sum())
    print(y_train.sum())
  
print(np.unique(y_train))
print(np.where(model.predict(x_train) > 0))
print(model.predict(x_train)[np.where(model.predict(x_train) > 0)])
print(scores)
print(np.where(y_train > 0))
print(y_train[np.where(model.predict(x_train) > 0)])

#print(scores.sum()/10)


  0%|                                                     | 0/2 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
 50%|██████████████████████▌                      | 1/2 [00:02<00:02,  2.55s/it]

0
27


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|█████████████████████████████████████████████| 2/2 [00:03<00:00,  1.92s/it]

6
270
[0 1 2 3]
(array([    6,   197,   685, 13504]),)
[1 1 3 1]
[[0.99929282]
 [0.98859669]]
(array([    1,   101,   127,   153,   169,   197,   215,   233,   238,
         371,   376,   515,   623,   645,   685,   704,   909,   995,
        1052,  1065,  1087,  1089,  1275,  1303,  1330,  1440,  1449,
        1456,  1586,  1698,  1729,  1730,  1758,  1779,  1898,  1997,
        1999,  2405,  2465,  2467,  2999,  3427,  3458,  3499,  3558,
        3561,  3612,  3639,  3763,  3764,  4314,  4433,  4434,  4511,
        4546,  4634,  4670,  5251,  5400,  5445,  5594,  5814,  6054,
        6087,  6169,  6198,  6202,  6216,  6245,  6246,  6253,  6270,
        6426,  6482,  6500,  6527,  6549,  6604,  6617,  6646,  6650,
        6655,  6682,  6726,  6757,  6803,  6923,  6954,  7007,  7051,
        7071,  7087,  7122,  7185,  7195,  7212,  7218,  7359,  7436,
        7512,  7625,  7629,  7728,  7816,  7979,  8021,  8108,  8109,
        8203,  8224,  8302,  8491,  8549,  8565,  8581,  8668,  8

## Part 2
### Question 1
Construct a multi-armed bandit algorithm such that

1. It is randomly initialized at first and selects **one** product out of $j$ available products.
2. It updates  $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$  over  time by observing the utility $\widehat{u}_{ij}$ of each product $j$ it selected in the past and selects new products


In [ ]:
#Hint: Try ridge regression on each arm separately,


def decicde():
    
    return 


def update_parameter():
    
    
    return 




### Question 2

 Draw 1000 random consumers from your data. For each consumer,  run your online learning algorithm for 100 steps. Note that this is a simulation process --- i.e., your algorithm itself does not know $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$, but can only observe the $\widehat{u}_{ij}$ for any product $j$ that the algorithm pulled (i.e., purchased).     
 For each randomly picked consumer $i$, compute the difference $\Delta_i$ between the  maximum utility $\max_j\widehat{u}_{ij}$ (i.e., consumer $i$'s  utility for her  favorite product) and the average utility that your algorithm
achieved at the 100th step. Compute the average of $\Delta_i$ over those 1000 consumers, and explain why there is such a difference.  

In [ ]:
def rewards_difference():
    
    
    return 



def simulation():
    
    
    return 




    



Explain why there is such a difference.

**Please input your answer in this cell:**